In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

In [2]:
train_df = pd.read_csv('./sh/train.csv')
test_df = pd.read_csv('./sh/train.csv')
train_df_all = pd.read_csv('./sh/countrywide_accident.csv')

In [3]:
train_df=pd.concat([train_df,train_df_all], axis=0)

In [4]:
test_df.head()

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


In [5]:
test_df['도로형태'].value_counts()

도로형태
단일로 - 기타           19485
교차로 - 교차로안          9931
교차로 - 교차로부근         5746
기타 - 기타             1986
교차로 - 교차로횡단보도내      1474
단일로 - 지하차도(도로)내      309
주차장 - 주차장            236
단일로 - 교량             232
단일로 - 고가도로위          138
단일로 - 터널              64
미분류 - 미분류              8
Name: count, dtype: int64

In [6]:
train_df['도로형태'].value_counts()

도로형태
단일로 - 기타           273386
교차로 - 교차로안         200543
교차로 - 교차로부근         96104
기타 - 기타             30521
교차로 - 교차로횡단보도내      21766
단일로 - 지하차도(도로)내      7091
단일로 - 교량             4411
주차장 - 주차장            3914
단일로 - 터널             2391
단일로 - 고가도로위          1909
미분류 - 미분류             336
단일로 - 철길건널목            12
Name: count, dtype: int64

In [7]:
train_df=train_df[train_df['도로형태']!="단일로 - 철길건널목"]

In [8]:
train_df=train_df[train_df['노면상태']!="해빙"]

In [9]:
def remove_outliers(df, column_name, threshold=1.5):
    """
    데이터프레임의 특정 열에서 IQR을 기반으로 이상치를 제거하는 함수.

    Parameters:
    - df: 데이터프레임
    - column_name: 이상치를 제거할 열의 이름
    - threshold: IQR을 계산할 때 사용할 상한값 (기본값은 1.5)

    Returns:
    - 이상치가 제거된 데이터프레임
    """
    # 특정 열의 값들을 추출
    column_values = df[column_name]

    # IQR 계산
    Q1 = column_values.quantile(0.25)
    Q3 = column_values.quantile(0.75)
    IQR = Q3 - Q1

    # 이상치의 상한과 하한 값 계산
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR

    # 이상치 제거
    df_no_outliers = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

    return df_no_outliers


In [10]:
#train_df=remove_outliers(train_df,'ECLO')

In [11]:
train_df['도로형태_1'] = train_df['도로형태'].str.split(' - ', expand=True)[0]
train_df['도로형태_2'] = train_df['도로형태'].str.split(' - ', expand=True)[1]
train_df.drop(['도로형태'], axis = 1, inplace = True)

test_df['도로형태_1'] = test_df['도로형태'].str.split(' - ', expand=True)[0]
test_df['도로형태_2'] = test_df['도로형태'].str.split(' - ', expand=True)[1]
test_df.drop(['도로형태'], axis = 1, inplace = True)

In [12]:
train_df[['시','구','동']] = train_df.시군구.str.split(' ',n=2,expand=True)
test_df[['시','구','동']] = test_df.시군구.str.split(' ',n=2,expand=True)

In [13]:
train_df['사고일시'] = pd.to_datetime(train_df['사고일시'])
train_df["사고년"] = train_df["사고일시"].dt.year
train_df["사고월"] = train_df["사고일시"].dt.month
train_df['사고일'] = train_df['사고일시'].dt.day
train_df['사고시간'] = train_df["사고일시"].dt.hour


test_df['사고일시'] = pd.to_datetime(test_df['사고일시'])
test_df['사고년'] = test_df['사고일시'].dt.year
test_df['사고월'] = test_df['사고일시'].dt.month
test_df['사고일'] = test_df['사고일시'].dt.day
test_df['사고시간'] = test_df['사고일시'].dt.hour

In [14]:
timeline = {5:'새벽',6:"새벽",7:"출근",8:"출근",18:"퇴근",19:"퇴근",20:"퇴근",
            9:"주간",10:"주간",11:"주간",12:"주간",13:"주간",14:"주간",15:"주간",16:"주간",
            17:"주간",21:"심야",22:"심야",23:"심야",0:"심야",1:"심야",2:"심야",3:"심야",4:"심야"}

season = {3:"봄", 4:"봄",5:"봄",
          6:"여름",7:"여름",8:"여름",
          9:"가을",10:"가을",11:"가을",
          1:"겨울",2:"겨울",12:"겨울"}


In [15]:
train_df['사고시간대'] = train_df['사고시간'].map(timeline)
train_df['계절'] = train_df["사고월"].map(season)
test_df['사고시간대'] = test_df['사고시간'].map(timeline)
test_df['계절'] = test_df["사고월"].map(season)

In [16]:
train_df['시'].value_counts()

시
경기도        161168
서울특별시      108389
대구광역시       39609
경상북도        39415
부산광역시       36243
경상남도        35626
전라남도        29371
충청남도        26776
충청북도        25917
인천광역시       25113
광주광역시       23429
대전광역시       22577
강원도         21194
전라북도        20172
제주특별자치도     12815
울산광역시       11980
세종특별자치시      2523
미분류            28
Name: count, dtype: int64

In [17]:
train_df.columns

Index(['ID', '사고일시', '요일', '기상상태', '시군구', '노면상태', '사고유형', '사고유형 - 세부분류',
       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',
       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',
       'ECLO', '도로형태_1', '도로형태_2', '시', '구', '동', '사고년', '사고월', '사고일', '사고시간',
       '사고시간대', '계절'],
      dtype='object')

In [18]:
train_df=train_df.loc[(train_df['시']=="울산광역시" ) | ( train_df['시']=="경상남도") | (train_df['시']=="경상북도") | (train_df['시']=="부산광역시") | (train_df['시']=="대구광역시") | (train_df['시']=="경상북도")].reset_index(drop = True)

#공휴일


In [19]:
# pip install holidays

In [20]:
import holidays
kr_holidays = holidays.KR()

In [21]:
train_df['공휴일'] = train_df['사고일시'].apply(lambda x: 'holiday' if x in kr_holidays else 'non-holiday')
test_df['공휴일'] = test_df['사고일시'].apply(lambda x: 'holiday' if x in kr_holidays else 'non-holiday')

In [22]:
holiday = {'토요일':"O", "일요일":"O","월요일":"X","화요일":"X","수요일":"X","목요일":"X","금요일":"X"}

In [23]:
train_df['주말여부']=train_df['요일'].map(holiday)
test_df['주말여부']=test_df['요일'].map(holiday)

In [24]:
train_df = train_df.drop(['ID','사고일시','시군구'], axis =1 )
test_df=test_df.drop(['ID','사고일시','시군구'], axis =1 )

In [25]:
train_df.isna().sum()

요일                0
기상상태              0
노면상태              1
사고유형              0
사고유형 - 세부분류       0
법규위반              0
가해운전자 차종          0
가해운전자 성별          0
가해운전자 연령          0
가해운전자 상해정도        0
피해운전자 차종       7697
피해운전자 성별       7697
피해운전자 연령       7696
피해운전자 상해정도     7697
사망자수              0
중상자수              0
경상자수              0
부상자수              0
ECLO              0
도로형태_1            0
도로형태_2            0
시                 0
구                 0
동                 0
사고년               0
사고월               0
사고일               0
사고시간              0
사고시간대             0
계절                0
공휴일               0
주말여부              0
dtype: int64

In [26]:
train_df=train_df.fillna('없음')

In [27]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162873 entries, 0 to 162872
Data columns (total 32 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   요일           162873 non-null  object
 1   기상상태         162873 non-null  object
 2   노면상태         162873 non-null  object
 3   사고유형         162873 non-null  object
 4   사고유형 - 세부분류  162873 non-null  object
 5   법규위반         162873 non-null  object
 6   가해운전자 차종     162873 non-null  object
 7   가해운전자 성별     162873 non-null  object
 8   가해운전자 연령     162873 non-null  object
 9   가해운전자 상해정도   162873 non-null  object
 10  피해운전자 차종     162873 non-null  object
 11  피해운전자 성별     162873 non-null  object
 12  피해운전자 연령     162873 non-null  object
 13  피해운전자 상해정도   162873 non-null  object
 14  사망자수         162873 non-null  int64 
 15  중상자수         162873 non-null  int64 
 16  경상자수         162873 non-null  int64 
 17  부상자수         162873 non-null  int64 
 18  ECLO         162873 non-null  int64 
 19  도로

In [28]:
train_df['기상상태'].value_counts()

기상상태
맑음    145669
비      12038
흐림      4210
기타       784
안개        93
눈         79
Name: count, dtype: int64

In [29]:
b_train_y = train_df['ECLO']
b_test_x = test_df
b_train_x = train_df[test_df.columns]
b_train_x['ECLO'] = train_df['ECLO']

In [30]:
b_train_x.columns.tolist()

['요일',
 '기상상태',
 '노면상태',
 '사고유형',
 '사고유형 - 세부분류',
 '법규위반',
 '가해운전자 차종',
 '가해운전자 성별',
 '가해운전자 연령',
 '가해운전자 상해정도',
 '피해운전자 차종',
 '피해운전자 성별',
 '피해운전자 연령',
 '피해운전자 상해정도',
 '사망자수',
 '중상자수',
 '경상자수',
 '부상자수',
 'ECLO',
 '도로형태_1',
 '도로형태_2',
 '시',
 '구',
 '동',
 '사고년',
 '사고월',
 '사고일',
 '사고시간',
 '사고시간대',
 '계절',
 '공휴일',
 '주말여부']

In [31]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import pandas as pd

class LinearRegressionWithEmbeddingMultiCategory(nn.Module):
    def __init__(self, num_numeric_features, category_dimensions, embedding_dim):
        super(LinearRegressionWithEmbeddingMultiCategory, self).__init__()

        # 여러 개의 임베딩 레이어
        self.embedding_layers = nn.ModuleList([
            nn.Embedding(num_categories, embedding_dim) for num_categories in category_dimensions
        ])
        self.batch_norm = nn.BatchNorm1d(num_numeric_features + len(category_dimensions) * embedding_dim)

        # 선형 회귀 레이어
        total_embedding_dim = len(category_dimensions) * embedding_dim
        self.linear_layer = nn.Linear(num_numeric_features + total_embedding_dim, 1)



    def forward(self, numeric_data, category_data_list):
        # 각 카테고리형 변수에 대한 임베딩
        embedded_category_data_list = [embedding_layer(category_data) for embedding_layer, category_data in zip(self.embedding_layers, category_data_list)]

        # 임베딩된 벡터들을 연결
        combined_embedding_data = torch.cat(embedded_category_data_list, dim=1)

        # 임베딩된 벡터와 수치형 데이터를 결합
        combined_data = torch.cat([numeric_data, combined_embedding_data], dim=1)

        # 선형 회귀 모델의 순전파
        output = self.linear_layer(combined_data)

        return output.squeeze()




In [32]:
class LinearRegressionDatasetMultiCategory(Dataset):
    def __init__(self, dataframe, numeric_columns, category_columns, target_column):
        self.numeric_data = dataframe[numeric_columns].values
        self.category_data_list = [dataframe[column].astype('category').cat.codes.values for column in category_columns]
        self.target_data = dataframe[target_column].values

    def __len__(self):
        return len(self.target_data)

    def __getitem__(self, idx):
        numeric_sample = torch.tensor(np.asarray(self.numeric_data[idx], dtype=np.float32), dtype=torch.float32)
        category_samples = [torch.tensor(category_data[idx], dtype=torch.long) for category_data in self.category_data_list]
        target_sample = torch.tensor(self.target_data[idx], dtype=torch.float32)
        return numeric_sample, category_samples, target_sample

In [33]:
train_list = b_train_x.columns.tolist()
remove_cat_list = ['요일',
 '기상상태',
 '노면상태',
 '사고유형',
 '도로형태_1',
 '도로형태_2',
 '시',
 '구',
 '동',
 '사고년',
 '사고월',
 '사고일',
 '사고시간대',
 '계절',
 '공휴일',
 '주말여부',
 'ECLO']
train_list = [i for i in train_list if i not in remove_cat_list]
print(train_list)

['사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', '사고시간']


In [34]:
# 데이터셋 및 데이터로더 생성
category_columns = test_df.columns.tolist()
numeric_columns = train_list
target_column = 'ECLO'

category_dimensions = [len(b_train_x[column].unique()) for column in category_columns]

dataset = LinearRegressionDatasetMultiCategory(b_train_x, numeric_columns, category_columns, target_column)
train_loader = DataLoader(dataset, batch_size=256, shuffle=True)


In [35]:
def rmsle(y_true, y_pred):
    y_true = torch.max(y_true.float(), torch.tensor(0.0))
    y_pred = torch.max(y_pred.float(), torch.tensor(0.0))
    squared_error = torch.square(torch.log1p(y_pred) - torch.log1p(y_true))
    return torch.sqrt(torch.mean(squared_error))

def loss_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

In [36]:
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LinearRegressionWithEmbeddingMultiCategory(
    num_numeric_features=len(numeric_columns),
    category_dimensions=category_dimensions,
    embedding_dim=50
).to(device)


# 손실 함수 및 옵티마이저 설정
criterion = loss_fn
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
# 훈련
epochs = 100
for epoch in tqdm(range(epochs)):
    total_loss = 0.0
    for numeric_data, category_data_list, target_data in train_loader:
        numeric_data , target_data = numeric_data.to(device) ,target_data.to(device)
        for i in range(len(category_data_list)):
            category_data_list[i] = category_data_list[i].to(device)
        # Forward pass
        output = model(numeric_data, category_data_list)

        # Compute loss
        loss = criterion(output, target_data)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        # Accumulate total loss
        total_loss += loss.item()

    # Print average loss during epoch
    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{epochs}, Average Loss: {average_loss}')



  0%|                                                        | 0/100 [00:00<?, ?it/s]


ValueError: could not convert string to float: '측면충돌'

In [37]:
b_test_x

,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,...,구,동,사고년,사고월,사고일,사고시간,사고시간대,계절,공휴일,주말여부
0,화요일,맑음,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,...,중구,대신동,2019,1,1,0,심야,겨울,holiday,X
1,화요일,흐림,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,...,달서구,감삼동,2019,1,1,0,심야,겨울,holiday,X
2,화요일,맑음,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,...,수성구,두산동,2019,1,1,1,심야,겨울,holiday,X
3,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,...,북구,복현동,2019,1,1,2,심야,겨울,holiday,X
4,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,...,동구,신암동,2019,1,1,4,심야,겨울,holiday,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,금요일,맑음,건조,차대차,측면충돌,신호위반,승용,여,52세,상해없음,...,수성구,수성동3가,2021,12,31,19,퇴근,겨울,non-holiday,X
39605,금요일,맑음,건조,차대차,측면충돌,안전거리미확보,승용,여,60세,상해없음,...,달서구,상인동,2021,12,31,19,퇴근,겨울,non-holiday,X
39606,금요일,맑음,건조,차대차,측면충돌,교차로운행방법위반,승용,남,60세,중상,...,달서구,월성동,2021,12,31,21,심야,겨울,non-holiday,X
39607,금요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,40세,상해없음,...,달서구,장동,2021,12,31,22,심야,겨울,non-holiday,X


In [ ]:
model.eval()
with torch.no_grad():
  pred = model(torch.tensor(b_test_x.values, dtype=torch.float32).to(device))

# 인코딩 (oversampling목표로 label encoding)

In [ ]:
test_x = test_df.copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [ ]:
pip install category_encoders

In [ ]:
from sklearn.preprocessing import LabelEncoder
from category_encoders.target_encoder import TargetEncoder

categorical_features = list(train_x.dtypes[train_x.dtypes == "object"].index)
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    le = TargetEncoder(cols=[i])
    train_x[i] = le.fit_transform(train_x[i], train_y)
    test_x[i] = le.transform(test_x[i])

display(train_x.head())
display(test_x.head())

In [ ]:
train_x.drop(['도로형태_1','시'],axis =1 , inplace=True)
test_x.drop(['도로형태_1','시'],axis =1 , inplace=True)

#딥러닝 파이토치 모델 생성을 목표로 (속도를 보장 못함) -> 공부필요

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset
import torch.nn.functional as F

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_x,train_y, test_size = 0.2, random_state = 48)

In [ ]:
X_train_torch=torch.Tensor(x_train.values)
X_test_torch=torch.Tensor(x_test.values)
y_train_torch=torch.Tensor(y_train.values)
y_test_torch=torch.Tensor(y_test.values)

In [ ]:
train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset = TensorDataset(X_test_torch, y_test_torch)


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=32,
                                          shuffle=True,
                                          )

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=32,
                                          shuffle=False,
                                          )

In [ ]:
def rmsle(y_true, y_pred):
    y_true = torch.max(y_true.float(), torch.tensor(0.0))
    y_pred = torch.max(y_pred.float(), torch.tensor(0.0))
    squared_error = torch.square(torch.log1p(y_pred) - torch.log1p(y_true))
    return torch.sqrt(torch.mean(squared_error))

def loss_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

def metric_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(15, 32)
        self.bn1 = nn.BatchNorm1d(num_features=32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(num_features=64)
        self.fc3 = nn.Linear(64,1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 및 데이터를 GPU로 이동
model = Model().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.01)

#스케쥴러 100 epoch 마다 10%만큼 학습률 감소
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

best_loss = 10 ** 9 # 매우 큰 값으로 초기값 가정
patience_limit = 20 # 몇 번의 epoch까지 지켜볼지를 결정
patience_check = 0 # 현재 몇 epoch 연속으로 loss 개선이 안되는지를 기록


model.train()  # 모델을 훈련 모드로 전환

num_epochs = 1000
for epoch in tqdm(range(num_epochs)):
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Forward pass
        y_pred = model(batch_X)

        # 손실 계산
        loss = loss_fn(y_pred, batch_y)

        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()


    # 로그 출력
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

  # 평가
    model.eval()  # 모델을 평가 모드로 전환

  # 데이터로더를 사용한 평가
    total_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            # Forward pass
            y_pred = model(batch_X)

           # 손실 계산
            total_loss += loss_fn(y_pred, batch_y)

            total_loss += loss.item()

    ### total_loss stopping 여부를 체크하는 부분 ###
        if total_loss > best_loss: # loss가 개선되지 않은 경우
            patience_check += 1

            if patience_check >= patience_limit: # early stopping 조건 만족 시 조기 종료
                break

        else: # loss가 개선된 경우
            best_loss = total_loss
            patience_check = 0

    average_loss = total_loss / len(test_loader)

print(f'Average Loss on Evaluation Data: {average_loss:.4f}')



In [ ]:
model.eval()
with torch.no_grad():
  pred = model(torch.tensor(test_x.values, dtype=torch.float32).to(device))

In [ ]:
pred